In [24]:
from os import walk
from os.path import join
import pandas as pd
import time
import numpy as np


import matplotlib.pyplot as plt

%matplotlib inline

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup

from wordcloud import WordCloud

from PIL import Image

<class 'ModuleNotFoundError'>: No module named 'wordcloud'

In [3]:
EXAMPLE_FILE = 'SpamData/01_Processing/practice_email.txt'
SPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_1'
SPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/spam_2'
NONSPAM_1_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_1'
NONSPAM_2_PATH = 'SpamData/01_Processing/spam_assassin_corpus/easy_ham_2'

EMAIL_TEXT_JSON_PATH = 'SpamData/01_Processing/email-text.json'
WORD_ID_FILE = 'SpamData/01_Processing/word-by-id.csv'


SPAM_CAT = 1
NONSPAM_CAT = 0
VOCABULARY_SIZE = 2500

TRAINING_DATA = 'SpamData/02_Training/train-data.txt'
TESTING_DATA = 'SpamData/02_Training/test-data.txt'

WHALE_FILE =  'SpamData/01_Processing/wordcloud_resources/whale-icon.png'
SKULL_FILE = 'SpamData/01_Processing/wordcloud_resources/skull-icon.png'

THUMBS_UP_FILE = 'SpamData/01_Processing/wordcloud_resources/thumbs-up.png'
THUMBS_DOWN_FILE = 'SpamData/01_Processing/wordcloud_resources/thumbs-down.png'
CUSTOM_FONT_FILE = 'SpamData/01_Processing/wordcloud_resources/OpenSansCondensed-Bold.ttf'

In [4]:
import sys
sys.getfilesystemencoding()

'utf-8'

In [5]:
def get_email_body(path):
    
    for root, dirname, filenames in walk(path):
        for file_name in filenames:
            filepath = join(root,file_name)

            stream = open(filepath, encoding='latin_1')
            is_body = False
            lines = []
            for line in stream:
                if is_body:
                    lines.append(line)
                elif line == '\n':
                    is_body=True

            stream.close()

            email_body = '\n'.join(lines)
            yield file_name, email_body


In [6]:
def dataframe_from_directory(path,classification):
    rows = []
    rows_name = []
    for file_name, email_body in get_email_body(path):
        rows.append({'MESSAGE' : email_body, 'CATEGORY' : classification})
        rows_name.append(file_name)
        
    return pd.DataFrame(rows,index=rows_name)
        
    

In [7]:
spam_emails =  dataframe_from_directory(SPAM_1_PATH,SPAM_CAT) # 1 : spam 0 : nonspam
spam_emails = spam_emails.append( dataframe_from_directory(SPAM_2_PATH,SPAM_CAT))
spam_emails.head()

<ipython-input-7-b37f582f5ebb>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  spam_emails = spam_emails.append( dataframe_from_directory(SPAM_2_PATH,SPAM_CAT))


""


In [8]:
spam_emails.shape

(0, 0)

In [9]:
non_spam_emails = dataframe_from_directory(NONSPAM_1_PATH,NONSPAM_CAT)
non_spam_emails = non_spam_emails.append(dataframe_from_directory(NONSPAM_2_PATH,NONSPAM_CAT))
print(non_spam_emails.shape)
non_spam_emails.head()

<ipython-input-9-f3343cc18b96>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  non_spam_emails = non_spam_emails.append(dataframe_from_directory(NONSPAM_2_PATH,NONSPAM_CAT))


(0, 0)


""


In [10]:
data = pd.concat([spam_emails,non_spam_emails])
print(f' shape of entire dataframe is {data.shape}')

 shape of entire dataframe is (0, 0)


In [11]:
data.head()

""


## Checking for null or missing values

In [12]:
data['MESSAGE'].isnull().values.any()

<class 'KeyError'>: 'MESSAGE'

In [13]:
#Removing system file entries (cmds files)
data = data.drop(['cmds'])

<class 'KeyError'>: "['cmds'] not found in axis"

In [14]:
data.shape 

(0, 0)

## Assigning document ID to each row 

In [15]:
document_ids = range(0,len(data.index))
data['DOC_ID'] = document_ids  # column to hold index 0-5796

In [16]:
data['FILE_NAME'] = data.index   #creating separate column to hold original file names
data.set_index('DOC_ID',inplace=True)  #changing index from filenales to DOC_ID
data.head()

,FILE_NAME
DOC_ID,


## storing emails data to json file

In [17]:
data.to_json(EMAIL_TEXT_JSON_PATH)

<class 'OSError'>: Cannot save file into a non-existent directory: 'SpamData/01_Processing'

## calculating amount of spam and legit emails

In [18]:
data['CATEGORY'].value_counts()

<class 'KeyError'>: 'CATEGORY'

In [19]:
amount_of_spam = data['CATEGORY'].value_counts()[1]
amount_of_ham = data['CATEGORY'].value_counts()[0]

category_names = ['spam' , 'legit emails']
sizes = [amount_of_spam ,amount_of_ham]

chart_colors = ['#b2ebf2','#e97171']

<class 'KeyError'>: 'CATEGORY'

## Visualizing spam and legit emails

In [20]:
plt.figure(figsize=(6,6),dpi = 100) # dpi: density of pixels per inch
plt.pie(sizes ,labels=category_names,textprops={'fontsize' : 14} ,startangle=90,colors=chart_colors,explode=[0,0.1],
        autopct='%1.0f%%')
plt.show()

<Figure size 600x600 with 0 Axes>

<class 'NameError'>: name 'sizes' is not defined

# pre processing with Natural laguage processing

### Text pre-processing

In [21]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error unknown url type:
[nltk_data]     https>


False

## Download NLTK resources (tokenizer and stopwords)

## Tokenizing and Removing stopwords

In [22]:
# Removing stopwords

stop_words = set(stopwords.words('english'))
stopwords

<class 'LookupError'>: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/pyodide/nltk_data'
    - '/nltk_data'
    - '/share/nltk_data'
    - '/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## Function to remove HTML tags and punctuation marks in emails. (cleaning)

In [23]:
# Defining function to perform all theese cleaning steps and also removing HtML tags

def clean_messages_no_html(message , stemmer = PorterStemmer() 
                   ,stop_words = set(stopwords.words('english'))):
    filtered_words = []
    
    soup = BeautifulSoup(message , 'html.parser')
    cleaned_text = soup.get_text()
    
    words = word_tokenize(cleaned_text.lower())
    for word in words:
        if word not in stop_words and word.isalpha(): #checking word in stopwords ? and removing punctuation
            filtered_words.append(stemmer.stem(word)) #stemming word
     
    
    
    return filtered_words

<class 'LookupError'>: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/pyodide/nltk_data'
    - '/nltk_data'
    - '/share/nltk_data'
    - '/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
#clean_messages_no_html(data.at[0,'MESSAGE'])

## Apply cleaning and tokenization to all emails

### slicing dataframes and series and creating sublists

In [ ]:
data[data['CATEGORY'] == 1].shape

In [ ]:
data[data['CATEGORY'] == 0].shape

In [ ]:
#Storing index of spam and ham messages in variables

doc_ids_ham = data[data['CATEGORY'] == 0].index  
doc_ids_spam = data[data['CATEGORY'] == 1].index

In [ ]:
%%time
nested_list = data['MESSAGE'].apply(clean_messages_no_html)  #List of lists

In [ ]:
nested_list.head()

In [ ]:
nested_list_spam = nested_list.loc[doc_ids_spam]
nested_list_ham = nested_list.loc[doc_ids_ham]

print(type(nested_list_ham))

In [ ]:
flat_list_spam = []
flat_list_ham = []

for sublist in nested_list_spam:
    for word in sublist:
        flat_list_spam.append(word)

        
         
for sublist in nested_list_ham:
    for word in sublist:
        flat_list_ham.append(word)

In [ ]:
print(type(flat_list_ham))

# Converting list to pandas series
normal_words = pd.Series(flat_list_ham).value_counts()
# print(f'total number of normal words : {normal_words.shape[0]}')
print(f'total number of unique normal words : {normal_words.shape[0]}')


spammy_words = pd.Series(flat_list_spam).value_counts()  # value_counts() holds only unique words
# print(f'total number of spammy words : {spammy_words.shape[0]}')
print(f'total number of unique spammy words : {spammy_words.shape[0]}')

In [ ]:
# Top most used words
print(f'Most used normal words: \n\n{normal_words[:10]}\n')
print(f'\nMost used spammy words: \n\n{spammy_words[:10]}\n')

## Creating wordcloud ( Highlights most occured words)

In [ ]:
# Generating text as string for spam messages
spam_str = ' '.join(flat_list_spam)

icon = Image.open(THUMBS_UP_FILE)
image_mask = Image.new(mode='RGB',size=icon.size,color=(255,255,255))
image_mask.paste(icon,box=icon)
rgb_array = np.array(image_mask)

word_cloud = WordCloud(mask=rgb_array,max_words=2000,background_color='white',max_font_size=300,
                       colormap='gist_heat',font_path=CUSTOM_FONT_FILE)
word_cloud.generate(spam_str.upper())


plt.figure(figsize=(16,8))
plt.imshow(word_cloud,interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Generating text as string for ham messages
ham_str = ' '.join(flat_list_ham)

icon = Image.open(THUMBS_DOWN_FILE)
image_mask = Image.new(mode='RGB',size=icon.size,color=(255,255,255))
image_mask.paste(icon,box=icon)
rgb_array = np.array(image_mask)

word_cloud = WordCloud(mask=rgb_array,max_words=2000,background_color='white',max_font_size=300,
                       colormap='gist_heat',font_path=CUSTOM_FONT_FILE)
word_cloud.generate(ham_str.upper())


plt.figure(figsize=(16,8))
plt.imshow(word_cloud,interpolation='bilinear')
plt.axis('off')
plt.show()

## Generate vocabulary and dictionary of words

In [ ]:
stemmed_nested_list = data['MESSAGE'].apply(clean_messages_no_html)  # nested list of words in each emails bodies
flat_stemmed_list = [item for sublist in stemmed_nested_list for item in sublist]

In [ ]:
unique_words = pd.Series(flat_stemmed_list).value_counts()
print(f' Total number of unique words : {unique_words.shape[0]}')
unique_words.head()

In [ ]:
frequent_words = unique_words[0:VOCABULARY_SIZE]
print( f'Most 10 frequently used words : \n {frequent_words[0:10]}')

## Creating vocacbulary DataFrame with WORD_ID

In [ ]:
word_ids = list(range(0,VOCABULARY_SIZE))
vocab = pd.DataFrame({'VOCAB_WORD':frequent_words.index.values})
print(type(vocab))
vocab.index.name= 'WORD_ID'
vocab.head()

## Save the vocabulary to csv file

In [ ]:
vocab.to_csv(WORD_ID_FILE,index_label=vocab.index.name,header=vocab['VOCAB_WORD'].name)

## Generate features and Sparse matrix


### Creating DataFrame with one wird per column

In [ ]:
stemmed_nested_list

In [ ]:
word_column_df = pd.DataFrame.from_records(stemmed_nested_list.to_list())
word_column_df.head()# converting pandas series to list of list

In [ ]:
word_column_df.shape

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(word_column_df, data['CATEGORY'], test_size = 0.3, random_state = 42)

In [ ]:
X_train.index.name = X_test.index.name = 'DOC_ID' 

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Making sparse matrix

In [ ]:
word_index = pd.Index(vocab['VOCAB_WORD'])
print(type(word_index))
print(len(word_index))
word_index

In [ ]:
def make_sparse_matrix(df, indexed_words, labels):
    '''
    returns sparse matrix as dataframe
    
    df : A dataframe with words in the column and document_id as index (X_train , X_test)
    indexed_words : index of words order by word_id
    labels : category as pd series (y_train,y_test)
    '''
    
    nr_rows = df.shape[0]
    nr_cols = df.shape[1]
    dict_list = [] # list of dictionaries
    word_set = set(indexed_words)
    
    for i in range(nr_rows):
        for j in range(nr_cols):
            word = df.iat[i,j]
            if word in word_set:
                doc_id = df.index[i]
                word_id = indexed_words.get_loc(word)
                category = labels.at[doc_id]
                
                item = {'LABEL' : category , 'DOC_ID' : doc_id,'WORD_ID' : word_id,'OCCURENCE' : 1}
                
                dict_list.append(item)
    return pd.DataFrame(dict_list)

In [ ]:
%%time
sparse_train_df= make_sparse_matrix(X_train,word_index,y_train)

In [ ]:
sparse_train_df.head()

In [ ]:
sparse_train_df['OCCURENCE'].isnull().any()

In [ ]:
sparse_train_df.shape

## Combine ocuurences with pandas groupby() method

In [ ]:
training_grouped_data = sparse_train_df.groupby(['DOC_ID','WORD_ID','LABEL']).sum()
training_grouped_data.head()

In [ ]:
training_grouped_data = training_grouped_data.reset_index()
training_grouped_data.head()

In [ ]:
training_grouped_data.shape

## Saving training data as .txt file

In [ ]:
#saving training data file

np.savetxt(TRAINING_DATA,training_grouped_data,fmt='%d')


In [ ]:
%%time
sparse_test_df= make_sparse_matrix(X_test,word_index,y_test)

In [ ]:
sparse_test_df.head()

In [ ]:
testing_grouped_data = sparse_test_df.groupby(['DOC_ID','WORD_ID','LABEL']).sum().reset_index()

In [ ]:
# saving testing data file

np.savetxt(TESTING_DATA,testing_grouped_data,fmt='%d')

In [ ]:
testing_grouped_data.columns

In [ ]:
sparse_test_df.shape